# Módulo 6 Actividad 2

## Intención del aprendizaje esperado:

2. Elaborar un modelo predictivo aplicando el algoritmo K-Means utilizando lenguaje Python para resolver un problema de clusterización.


## Ejercicios Planteados


*Clasificación de Clientes según su personalidad.*

**Luego de haber cerrado su cápsula de aprendizaje de clasificación, se le ha determinado que averigüe los intereses que pueden tener los clientes de la empresa KEPPLER, esto es necesario, para que se entregue la información al departamento de marketing y puedan realizar campañas fidelizada a sus clientes.**
**En el archivo clientes.cvs encontrará el registro de cada uno de ellos, además el archivo contiene 9 categorías -actividades laborales- que son:**

● Actor/actriz

● Cantante

● Modelo

● Tv, series

● Radio

● Tecnología

● Deportes

● Política

● Escritor

**1. Realice un resumen estadístico de sus datos.**

usuario (el nombre en Twitter)

“op” = Openness to experience – grado de apertura mental a nuevas experiencias, curiosidad, arte

“co” =Conscientiousness – grado de orden, prolijidad, organización

“ex” = Extraversion – grado de timidez, solitario o participación ante el grupo social

“ag” = Agreeableness – grado de empatía con los demás, temperamento

“ne” = Neuroticism, – grado de neuroticismo, nervioso, irritabilidad, seguridad en sí mismo.

Wordcount – Cantidad promedio de palabras usadas en sus tweets

Categoria – Actividad laboral del usuario (actor, cantante, etc.)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
 
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

import warnings
warnings.filterwarnings("ignore")

clientes = pd.read_csv('clientes.csv')

print(clientes.size)
print('\n' + '=' * 80)
clientes.info()
print('\n' + '=' * 80)
print(clientes.describe())
print('\n' + '=' * 80)
print(clientes['categoria'].value_counts())




**2. Entregue gráficos que acompañen los hallazgos encontrados en el punto 1.**

In [ ]:
clientes.hist()
plt.show()


EL análisis exploratorio no nos permite obtener ninguna información que nos dé indicios de algún patrón de comportamiento.  Las métricas de las variables son desbalanceadas por lo que se supone que no están estandarizadas pero tampoco se puede asegurar.  Vemos en los gráficos que la distribucuón pareciera no marcar tendencia segura en algo tampoco ya que muchos de los gráficos muestran comportamientos diferentes a la distribución normal o desbalanceados.

**3. Encuentre una relación o grafique las variables: op, ex y ag, además de las categorías.**


In [ ]:
variables_estudio = ['op', 'ex', 'ag', 'categoria']
clientes[variables_estudio].corr()

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.axes(projection='3d')
ax.scatter(clientes['op'], clientes['ex'], clientes['ag'], c=clientes['categoria'], cmap='viridis')
ax.set_xlabel('op')
ax.set_ylabel('ex')
ax.set_zlabel('ag')
plt.title('Relación entre op, ex y ag por categoría laboral')
plt.show()

In [ ]:

sns.pairplot(clientes.dropna(), hue='categoria',size=4,vars=["op","ex","ag"],kind='scatter', palette='dark')
plt.show()

Mi interpretación tanto de los datos de la matriz de correlación como los gráficos, histogramas y gráficos de dispersión, no se observa una tendencia clara sobre la distribución de las categorías ni alguna relación observable a simple vista.

**4. Determine cuántos k o clúster son adecuados para esta consulta.**

In [ ]:
X = clientes[['op', 'ex', 'ag']]
y = clientes['categoria']

distorsion = []
K = range(1, 20)
for k in K:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    distorsion.append(kmeans.inertia_)

diferencias = [distorsion[i] - distorsion[i-1] for i in range(1, len(distorsion))]

print(' Diferencias Distorsión por K '.center(70, '=') + '\n')
for i, diff in enumerate(diferencias, start=1):
    print(f'Distorsión K{i} = {distorsion[i]:.2f} '.ljust(35, '.') + f' Diferencia {i} K: {diff:.2f}'.rjust(35, '.'))   

plt.figure(figsize=(8, 6))
plt.plot(K, distorsion, marker='.', color='gray')
plt.xlabel('Número de Clústeres (k)')
plt.ylabel('Distorsión')
plt.title('Distorsión vs. Número de Clústeres')
plt.xticks(range(1,21))
plt.xlim(0, 20)
plt.grid()
plt.show()

En consideración del reporte de diferencias de Distorsión por K y el gráfico, optaría por un valor de 5 clúster que es el momento en que la optimización que obtiene respecto al valor siguiente de K no es tan significativa en terminos de diferencia.  Me parece que representa mejor que 4 o 6 Ks siendo además un valor que ronda la media de la variable categoría.



**5. Realice un modelo de kmeans con los cluster definidos en el punto 4.**


In [ ]:
numero_clusters = 5

modelo_kmeans = KMeans(n_clusters=numero_clusters, random_state=42)
modelo_kmeans.fit(X)

clientes['cluster'] = modelo_kmeans.labels_
centroides_clusters = modelo_kmeans.cluster_centers_

numero_clusters_4 = 4

modelo_kmeans_k4 = KMeans(n_clusters=numero_clusters_4, random_state=42)
modelo_kmeans_k4.fit(X)

clientes['4cluster'] = modelo_kmeans_k4.labels_
centroides_clusters_4 = modelo_kmeans_k4.cluster_centers_


**6. Entregue una representación gráfica de los cluster.**

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5), subplot_kw={'projection': '3d'})

axs[0].scatter(clientes['op'], clientes['ex'], clientes['ag'], c=clientes['cluster'], cmap='tab10', s=20)
axs[0].scatter(centroides_clusters[:, 0], centroides_clusters[:, 1], centroides_clusters[:, 2], c='red', marker='*', s=200)
axs[0].set_xlabel('op')
axs[0].set_ylabel('ex')
axs[0].set_zlabel('ag')
axs[0].set_title('KMeans 5K Clustering en 3D')


axs[1].scatter(clientes['op'], clientes['ex'], clientes['ag'], c=clientes['4cluster'], cmap='tab10', s=20)
axs[1].scatter(centroides_clusters_4[:, 0], centroides_clusters_4[:, 1], centroides_clusters_4[:, 2], c='red', marker='*', s=200)
axs[1].set_xlabel('op')
axs[1].set_ylabel('ex')
axs[1].set_zlabel('ag')
axs[1].set_title('KMeans 4K Clustering en 3D')

plt.show()

fig1, axes1 = plt.subplots(1, 2, figsize=(12, 5))

axes1[0].scatter(clientes['op'], clientes['ex'], c=clientes['cluster'], cmap='tab10', s=20, alpha=0.6)
axes1[0].scatter(centroides_clusters[:, 0], centroides_clusters[:, 1], c='red', marker='*', s=200)
axes1[0].set_xlabel('op')
axes1[0].set_ylabel('ex')
axes1[0].set_title('KMeans 5K Clustering op-ex')

axes1[1].scatter(clientes['op'], clientes['ex'], c=clientes['4cluster'], cmap='tab10', s=20, alpha=0.6)
axes1[1].scatter(centroides_clusters_4[:, 0], centroides_clusters_4[:, 1], c='red', marker='*', s=200)
axes1[1].set_xlabel('op')
axes1[1].set_ylabel('ex')
axes1[1].set_title('KMeans 4K Clustering en op-ex')

plt.show()


fig2, axes2 = plt.subplots(1, 2, figsize=(12, 5))

axes2[0].scatter(clientes['op'], clientes['ag'], c=clientes['cluster'], cmap='tab10', s=20, alpha=0.6)
axes2[0].scatter(centroides_clusters[:, 0], centroides_clusters[:, 2], c='red', marker='*', s=200)
axes2[0].set_xlabel('op')
axes2[0].set_ylabel('ag')
axes2[0].set_title('KMeans 5K Clustering op-ag')

axes2[1].scatter(clientes['op'], clientes['ag'], c=clientes['4cluster'], cmap='tab10', s=20, alpha=0.6)
axes2[1].scatter(centroides_clusters_4[:, 0], centroides_clusters_4[:, 2], c='red', marker='*', s=200)
axes2[1].set_xlabel('op')
axes2[1].set_ylabel('ag')
axes2[1].set_title('KMeans 4K Clustering en op-ag')

plt.show()

fig3, axes3 = plt.subplots(1, 2, figsize=(12, 5))

axes3[0].scatter(clientes['ex'], clientes['ag'], c=clientes['cluster'], cmap='tab10', s=20, alpha=0.6)
axes3[0].scatter(centroides_clusters[:, 1], centroides_clusters[:, 2], c='red', marker='*', s=200)
axes3[0].set_xlabel('op')
axes2[0].set_ylabel('ag')
axes3[0].set_title('KMeans 5K Clustering ex-ag')

axes3[1].scatter(clientes['ex'], clientes['ag'], c=clientes['4cluster'], cmap='tab10', s=20, alpha=0.6)
axes3[1].scatter(centroides_clusters_4[:, 1], centroides_clusters_4[:, 2], c='red', marker='*', s=200)
axes3[1].set_xlabel('ex')
axes3[1].set_ylabel('ag')
axes3[1].set_title('KMeans 4K Clustering en ex-ag')

plt.show()



**7. A qué conclusión llega y cuál es su recomendación como experto.**

Probado el modelo con 4 y 5 K para poder contrastar, se llega a la conclusión que las dos cantidades de cluster permiten separaciones útiles para el análisis.  Desde el punto de vista gráfico, los clúster muestran mayor distancia intercluster en el modelo de 4 K motivo por lo cual yo me inclinaría por esta cantidad de clúster.  Recordemos que K Means es un algoritmo para clasificación que actúa iterando el promedio de las distancias respecto a una cantidad de centroides que estimamos previamente, esto agrupa los eventos dependiendo si cumplen con ciertas característica y los etiqueta con un clúster.

Desde mi perspectiva como experto, digo que en base a estos datos y clasificaciones que nos da el modelo podemos realizar diverso tipo de observaciones que quizás puedan entregar información relevante a una empresa de marketing, por ejemplo.

Podríamos generar una lista de personas por clúster y estudiar alguna relación que hayamos pasado por alto en el pre-análisis, podríamos identificar la cantidad de categorías por clúster y su valor, etc.


In [ ]:
usuario_por_cluster = ['usuario', 'categoria', '4cluster']
clientes_ejemplo = clientes[usuario_por_cluster]

cluster_0 = clientes_ejemplo[clientes_ejemplo['4cluster'] == 0]
cluster_0



In [ ]:
fig = plt.figure(figsize=(5, 4))

sns.histplot(data=cluster_0, x='categoria')

plt.title('Distribución de categorías en cluster  0', fontdict={'fontweight':'bold'}, pad=10)
plt.ylabel('Frecuencia')

plt.show()